In [21]:
import geopandas as gpd
import osmnx as ox
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt

In [12]:
place_name="Paris"
path="Y:\RECHERCHE ET DEV\\10_Modèle d'attractivite des rues\\04_données clients (source)\osm\\"+place_name.lower()


In [7]:
contour_iris=gpd.read_file(r'Y:\REFERENTIEL DATA\RP 2022 FRANCE\CARTOGRAPHIE\Cartographie_France_IRIS_2019\CONTOURS-IRIS_2-1_SHP_LAMB93_FXX-2019\CONTOURS-IRIS.shp').to_crs('EPSG:3857')

In [13]:
building=pd.read_pickle(path+r"\raw data\building.pkl").to_crs('EPSG:3857')

In [48]:
#building_no_na : buildings with field building:levels is not nan
building_no_na=building[~building['building:levels'].isna()]
building_na=building[building['building:levels'].isna()]

In [49]:
building_no_na['building:levels']=building_no_na['building:levels'].apply(lambda x:float(x.split(";")[0]))

In [50]:
train,test=train_test_split(building_no_na,random_state=123)

# Mode

In [123]:
mode=train['building:levels'].mode().values[0]
print(mode)
print("train error is :",(np.abs(train['building:levels']-mode)).sum()/len(train))
print("test error is :",(np.abs(test['building:levels']-mode)).sum()/len(test))

7.0
train error is : 2.1906825449332894
test error is : 2.2176314918954683


# Mean

In [125]:
mean=train['building:levels'].mean().round()
print(mean)

print("train error is :",(np.abs(train['building:levels']-mean)).sum()/len(train))
print("test error is :",(np.abs(test['building:levels']-mean)).sum()/len(test))

5.0
train error is : 2.1130995699636124
test error is : 2.1407542176645715


# KNN-mode

**Idée**:
Pour chaque batimenet , chercher les K batiments les plus proches avec une distance inferieur à 100 pour éviter les cas où le batimenets est vraiment loin des autres batiments. 
Prendre le nombre d'étage le plus occurent dans ces K proches Batiments.
Si y' a aucun batimenet dans l'approximité (distance>100) considere le nombre d'étage le plus occurenet dans l'IRIS .

In [126]:
iris_data=pd.DataFrame([],columns=["code_iris","average_level"])
k=10
def func(row):

    i=pd.DataFrame(row).transpose().index[0]
    distances=building_no_na.distance(row['geometry']).sort_values()
    distances=distances[distances<100]

    distances=distances.drop(i)

    level=building_no_na.loc[distances.index].iloc[:k]['building:levels'].mode().to_numpy()

    if len(level)>0:
        return level[0]
    else:
        #get all the buildings of the iris
        iris_building=contour_iris[contour_iris.intersects(row['geometry'])].iloc[0]
        #when we calculate the mode level of the buildings in the iris we store it in a dataframe 
        if iris_building ["CODE_IRIS"] in iris_data["code_iris"].to_list():
            l=iris_data[iris_data["code_iris"]==iris_building ["CODE_IRIS"]]['average_level'].iloc[0]
            return l
        else:
            buildings_in_iris=building_no_na[building_no_na.intersects(iris_building["geometry"])]
            level_in_iris=buildings_in_iris['building:levels'].mode().to_numpy()[0]

            
            iris_data.loc[len(iris_data)]=[iris_building['CODE_IRIS'],level_in_iris]
            return level_in_iris

In [127]:
import time
start=time.time()
levels=building_no_na.apply(lambda x:func(x),axis=1)
time.time()-start

15891.224867105484

In [129]:
y=np.array([float(str(i).split(';')[0]) for i in sample['building:levels'].to_numpy()])
y_pred=np.array([float(str(i).split(';')[0]) for i in levels])

In [131]:
print("The error is :"+str(np.sum(np.abs(y-y_pred))/len(y)))

The error is :2.6198478332782003
